# DPML | Latency Replay

In this notebook, we investigate the reproducibility of transformation sequences captured by `dpml`.

## Load Dependencies

In [1]:
from lineage import LeBatch

from sibyl import *
from datasets import load_dataset

import time
from tqdm.notebook import tqdm

## Create Datasets

In [2]:
dataset = load_dataset("glue", "sst2", split="train[:100]")
dataset = dataset.rename_column('sentence', 'text')

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


## Replay Test

### Routine to be Tracked

In [3]:
transform = SibylTransformer("sentiment", num_INV = 2, num_SIB = 2)

In [6]:
text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []

batch_size= 10

startTime = time.perf_counter()
for i in tqdm(range(0, len(label), batch_size)):
    text_batch = text[i:i+batch_size]
    label_batch = label[i:i+batch_size]
    batch = (text_batch, label_batch)
    new_records = LeBatch(batch).apply(transform)
print('Elapsed time: {:6.3f} seconds'.format(time.perf_counter() - startTime))

  0%|          | 0/10 [00:00<?, ?it/s]

Elapsed time: 39.337 seconds


In [7]:
new_records

[<LeRecord:
 	 text="that 's so sloppily written and cast that you can not believe anyone more central to the creation of bugsy than the caterer",
 	 target="[0.75, 0.25]",
 	 le_attrs={'transformation_provenance': <TransformationProvenance: {(0, "{'class': 'SibylTransformer', 'task': 'sentiment', 'num_classes': 2, 'multiplier': 1, 'num_INV': 2, 'num_SIB': 2}")}>, 'feature_provenance': <FeatureProvenance[edit_seq] set()>, 'granularity': 'word'}>,
 <LeRecord:
 	 text="alternating between facetious comic parody and pulp melodrama , this smart-aleck movie ... tosses around some intriguing questions about the difference between human and android life",
 	 target="[0.25, 0.75]",
 	 le_attrs={'transformation_provenance': <TransformationProvenance: {(0, "{'class': 'SibylTransformer', 'task': 'sentiment', 'num_classes': 2, 'multiplier': 1, 'num_INV': 2, 'num_SIB': 2}")}>, 'feature_provenance': <FeatureProvenance[edit_seq] set()>, 'granularity': 'word'}>,
 <LeRecord:
 	 text="strung-together mo